### Forecast de demanda: Facturapp

#### Conexión de datos

In [35]:
import pandas as pd

# Rutas a los archivos CSV
clientes_path = 'C:/Others/Emprendiendo/Agencia/Bummi ai/GitHub Bummi/forecast_facturapp/csv/clientes.csv'
productos_path = 'C:/Others/Emprendiendo/Agencia/Bummi ai/GitHub Bummi/forecast_facturapp/csv/productos.csv'
ventas_path = 'C:/Others/Emprendiendo/Agencia/Bummi ai/GitHub Bummi/forecast_facturapp/csv/ventas.csv'

# Leer los archivos CSV
clientes_df = pd.read_csv(clientes_path)
productos_df = pd.read_csv(productos_path)
ventas_df = pd.read_csv(ventas_path)

In [36]:
clientes_df.head()

,id,razon_soc,nombre_fantasia,pais,departamento,ciudad,activo,tipo_cliente
0,3,DELICIAS SANTI,NaN,URUGUAY,CERRO LARGO,MELO,N,NaN
1,9,ALICIA PEREIRA,NaN,URUGUAY,CERRO LARGO,MELO,N,NaN
2,10,CHIVETERIA ROXANA,NaN,URUGUAY,CERRO LARGO,MELO,N,NaN
3,13,AUTOSERVICE Y PANADERIA AUCHI,NaN,URUGUAY,CERRO LARGO,MELO,N,NaN
4,15,RESTAURANT LA VASCA,NaN,URUGUAY,CERRO LARGO,MELO,N,NaN


In [37]:
productos_df.head()

,codigo_producto,descripcion_producto,costo_producto,precio_producto,stock_actual_producto,unidad_de_medida_producto,proveedor_producto,ramo_producto,familia_producto,grupo_producto,marco_producto
0,404,MANTECA DOS AROLLOS KG,180.000,216.00,0.00,Uni,Proveedor de la Empresa,Sin asignar,ISSUE,TINTAS,NaN
1,3,COMPRA EXENTO IVA,0.000,0.00,12.47,Uni,Proveedor de la Empresa,VACIO2,DANICA,COMPRAS,NaN
2,5932,ALIMENTO SPECIAL DOG CARNE ADULTO 20KG,27.080,33.85,0.00,Uni,MANIZAL S.A,ALIMENTO MASCOTAS,PERROS,Sin asignar,SPECIAL DOG
3,250401,ENCENDEDOR INMINENT CLASICO (DISP 50 UNI) (20 ...,801.786,1002.00,1.00,Uni,MABLE S.A,HOGAR,VARIOS,Sin asignar,INMINENT
4,1077,BANDEJA BOLW 4 445BN,0.381,0.50,366.00,Uni,NaN,EMPAQUES,BANDEJA,Sin asignar,NaN


In [38]:
ventas_df.head()

,codigo_producto,moneda_venta,id_cliente,fecha_venta,monto_ventas_producto,cantidad_ventas_producto
0,0,$,1241,2022-12-30,0.0,1.0
1,0,$,1258,2022-12-16,0.0,1.0
2,0,$,1201,2024-01-30,0.0,1.0
3,0,$,1260,2023-03-01,0.0,1.0
4,0,$,1403,2023-08-15,0.0,1.0


#### Benchmark: ARIMA

#### Opción 1: Prophet

#### Opción 2: skforecast

### Opciones avanzadas

#### Opción avanzada 1: Temporal Fusion Transformers (TFT)

#### Opción avanzada 2: DeepAR (a través de GluonTS)

#### Opción avanzada 3: TimeGPT